In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [127]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 0.5
decimate = 10
drive_freq = 139
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
        
    freq2 = np.fft.rfftfreq(len(bb.other_data[5]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.other_data[3])*np.sqrt(2)*100)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-1000000, 10000000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     print(m1_tmp.values[2]-phi_tmp2)
    return m1_tmp.values[0]

In [128]:
func1_scale(0)

[ 6.1060245] 46.808918858


-6.595881897141226

In [129]:
step_cal_dir = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/'
max_file = 500

In [120]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

175


In [130]:
amps2 = []
for i in range(num_calib_files):
    print(i,'/',len(step_cal_files))
    amps2.append(func1_scale(i))

0 / 175
[ 6.1060245] 46.808918858
1 / 175
[ 5.70585241] 46.8139478139
2 / 175
[ 2.8430897] 46.8069680452
3 / 175
[ 4.6126093] 46.8089831696
4 / 175
[ 5.84814858] 46.807126936
5 / 175
[ 2.40443585] 46.8099933925
6 / 175
[ 5.52233212] 46.808383493
7 / 175
[ 5.21757971] 46.8093727407
8 / 175
[ 4.53555238] 46.8131521942
9 / 175
[ 5.22397503] 46.814888696
10 / 175
[ 4.85839615] 46.810626314
11 / 175
[ 4.83988584] 46.8071779938
12 / 175
[ 2.45546817] 46.8172939223
13 / 175
[ 0.04294623] 46.8125752567
14 / 175
[ 3.39915234] 46.8096528553
15 / 175
[ 2.0269053] 46.8155759051
16 / 175
[ 0.32479113] 46.8194221025
17 / 175
[ 5.75815931] 46.8188989916
18 / 175
[ 1.46080029] 46.8134821288
19 / 175
[ 5.12523719] 46.821452674
20 / 175
[ 2.05522277] 46.8145898035
21 / 175
[ 5.54829133] 46.8216801948
22 / 175
[ 1.05852631] 46.8237787287
23 / 175
[ 1.42242133] 46.8167774553
24 / 175
[ 4.95636114] 46.8229808012
25 / 175
[ 0.27998662] 46.8211623108
26 / 175
[ 2.86058377] 46.8240369047
27 / 175
[ 2.2362994]

In [135]:
# amps = np.array(amps)
amps2 = np.array(amps2)
# amps1 = np.array(amps1)
_,ax = plt.subplots(figsize=(9.5,4))
# scale = 375

# Perrinf
# ax.scatter(range(len(amps3)), amps3/scaleY, label='fit phase', alpha=0.3)
ax.scatter(range(len(amps2)), amps2/scaleY, label='fit phase')
# ax.scatter(range(len(amps)), amps/scaleY, label='fit phase', alpha=0.4)
# ax.scatter(range(len(amps1)), amps1/scale, label='fit phase', alpha=0.8)

# ax.set(xlabel='dataset', ylabel='electron charges', ylim=(-3,1), xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
# scale = -1*np.abs(np.mean(amps3[200:250]))/2
# scale = -1*np.abs(np.mean(amps[230:240]))/3
# scale = -1*np.abs(np.mean(amps[97:118]))/4
scaleY = np.abs(np.mean(amps2[30:90]))
print("Scales are ", scaleY)
bb = step_file_objs[-1]
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage)

Scales are  83.249919569
voltage:  24.0265126779


In [134]:
83.24/98, 83.24/98*3.3

(0.8493877551020408, 2.8029795918367344)

In [585]:
bb = step_file_objs[-1]
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage11 = np.std(xx2)*np.sqrt(2)*100
xx2 = signal.filtfilt(b, a, bb.other_data[2])
voltage22 = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage11, voltage22)
[np.std(bb.other_data[i])*100 for i in range(7)]

voltage:  9.56767040051 9.35558843601


[61.757437999298993,
 6.9271469440944031,
 6.7703950401382524,
 0.070769370716167054,
 0.057274780609570891,
 0.065717635375355604,
 0.060547750817076931]

In [17]:
scaleY = 97.9992158717
scaleZ = scaleY*(418+627)/(392+406)
scaleX = scaleY*(421+368)/(392+406)
scaleX, scaleY, scaleZ

(96.89396155735751, 97.9992158717, 128.33230649865476)

### Look at neutrality data

In [124]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
def get_amp(i):
    
    fname = r'/data/old_trap/20211001/bead1/neutrality5/Y_175Vpp_alternate_139Hz_spin_XZ_77kHz_no_delay_no_height_fb/Y_175Vpp_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/Y_50Vpp_'+str(i)+'.h5'

    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[5+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

#     fit_kwargs = {'A': 0.23, 'f': drive_freq, 'phi': 0, 
#                   'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
#                   'limit_A': [0, 1], 'fix_f': True, 'fix_phi': False,
#                   'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
#     xx2 = signal.filtfilt(b, a, bb.other_data[5+jj])[::decimate]    
#     m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     phi_tmp = m1_tmp.values[2]

#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    if jj==0:
        phase_diff = 0.12
    else:
        phase_diff = 0.05
            
#     fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
#                   'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
#                   'limit_A': [-100000, 100000], 'limit_sigma':[100,30000], 'fix_phi': True, 'fix_f': True,
#                   'print_level': 0}
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}
    
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = drive_freq*2
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+np.pi/2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    #     print(m1_tmp.values)
    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.fval

In [126]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
def get_ampZ(i):
    
    fname = r'/data/old_trap/20211001/bead1/neutrality11/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_200Vpp_delay_6sec_with_height_fb/Z_200Vpp_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20211001/bead1/both_electrodes/Z_139Hz_20Vpp/Z_139Hz_20Vpp_'+str(i)+'.h5'

    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[1+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    #     fit_kwargs = {'A': 0.23, 'f': drive_freq, 'phi': 0, 
#                   'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
#                   'limit_A': [0, 1], 'fix_f': True, 'fix_phi': False,
#                   'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
#     xx2 = signal.filtfilt(b, a, bb.other_data[5+jj])[::decimate]    
#     m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     phi_tmp = m1_tmp.values[2]

#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    if jj==0:
        phase_diff = 4.8
    else:
        phase_diff = 4.8
            
#     fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
#                   'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
#                   'limit_A': [-100000, 100000], 'limit_sigma':[100,3000000], 'fix_phi': True, 'fix_f': True,
#                   'print_level': 0}
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}
    
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = drive_freq*2
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff+np.pi/2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-1000000, 1000000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.values[3], m1_tmp.fval

In [125]:
i=2500
get_ampZ(i)[0]

-1.537964620962157

In [98]:
i = 0
fname = r'/data/old_trap/20211001/bead1/neutrality11/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_200Vpp_delay_6sec_with_height_fb/Z_200Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
print("voltage: ", voltage, np.mean(bb.other_data[1])*100, np.mean(bb.other_data[2])*100)
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
# voltage_200_1 = np.std(xx2)*np.sqrt(2)*100
xx3 = signal.filtfilt(b, a, bb.other_data[2])
voltage_200_2 = np.std(xx3)*np.sqrt(2)*100
print("voltage: ", voltage_200_1, voltage_200_2)

voltage:  24.0265126779 -0.578471351186 -0.592051270314
voltage:  190.934485622 187.927740957


In [82]:
fname = r'/data/old_trap/20211001/bead1/single_electrode_20211013/Z_plus_139Hz_50Vpp/Z_50Vpp_0.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage_50_single_plus = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage_50_single_plus)

fname = r'/data/old_trap/20211001/bead1/single_electrode_20211013/Z_minus_139Hz_50Vpp/Z_50Vpp_0.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[2])
voltage_50_single_minus = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage_50_single_minus)

voltage:  47.4463804615
voltage:  46.7024118353


In [83]:
voltage_50_single_minus/voltage_50_single_plus, voltage_200_2/voltage_200_1

(0.98431980229189575, 0.98425247982313169)

In [22]:
voltage,  voltage_175_1

(24.026512677896338, 161.02821043101855)

In [184]:
# amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay = []
# for i in range(1000):
#     print(i,'/500')
#     try:
#         amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay.append(get_amp(i))
#     except:
#         print("******")
# amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay = np.array(amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay)

In [24]:
def get_amp_wraper(i):
    try:
        return get_amp(i)
    except:
        return 0,0,0,0
    
def get_amp_wraperZ(i):
    try:
        return get_ampZ(i)
    except:
        return 0,0,0,0

In [48]:
amps_spin_xz_117kHz_y_alternate_200Vpp_redo10 = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraper)(i) for i in tqdm(range((1000)))))


100%|██████████| 1000/1000 [00:03<00:00, 301.81it/s]

In [25]:
amps_spin_xz_117kHz_y_both_175Vpp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i) for i in tqdm(range((220)))))


100%|██████████| 220/220 [00:11<00:00, 18.41it/s]

In [1502]:
i=0; j=1
_,ax = plt.subplots()
dat = amps_spin_xz_117kHz_y_both_175Vpp[::2]/scaleZ*voltage/voltage_200_1
dat1 = amps_spin_xz_117kHz_y_alternate_200Vpp_redo9[1::2]/scaleZ*voltage/voltage_200_1
ax.scatter(dat[:,i], np.abs(dat[:,j]))
ax.scatter(dat1[:,i], np.abs(dat1[:,j]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [314]:
[i for i,bb in enumerate(amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate) if bb[2]>20000]

[692]

In [316]:
amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate_2 = list(amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate)
amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate_2.pop(692)
amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate_2.pop(692)
amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate = np.array(amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate_2)

In [52]:
i=0; j=3
_,ax = plt.subplots()
dat = amps_all[::2]/scaleZ*voltage/voltage_150_1
ax.scatter(dat[:,i], np.abs(dat[:,j]))
dat1 = amps_all[1::2]/scaleZ*voltage/voltage_150_1
ax.scatter(dat1[:,i], np.abs(dat1[:,j]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [530]:
voltage= 25

In [39]:
etha = 1/scaleZ*327*1.6e-19*voltage
to_epsilon = 1/(2*327*1.6e-19*voltage_200_1*(418)/(418+627))

In [531]:
j = 0
data = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[::2,j]/scaleY*voltage/voltage_175_1
data2 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[1::2,j]/scaleY*voltage/voltage_175_1
data3 = data-0.96*data2

data_1 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[::2,1]/scaleY*voltage/voltage_175_1
data2_1 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[1::2,1]/scaleY*voltage/voltage_175_1
data3_1 = data_1+data2_1

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='175Vpp-y-plus, xz-spin-100Vpp-77kHz')
ax.scatter(range(len(data2)), data2, label='175Vpp-y-minus, xz-spin-100Vpp-77kHz', alpha=0.6)
# ax.scatter(range(len(data3)), data3, label='100Vpp-y-diff, xz-spin-100Vpp-77kHz', alpha=0.6)

# ax.scatter(range(len(data4)), data4, label='20V-x, xy-spin-30V, w/o height fb')
ax.plot(np.arange(0,250), np.arange(0,250)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)))
print('mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))
print('mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))
print('mean, std: ', np.mean(data_1), np.std(data_1)/np.sqrt(len(data_1)))
print('mean, std: ', np.mean(data2_1), np.std(data2_1)/np.sqrt(len(data2_1)))
print('mean, std: ', np.mean(data3_1), np.std(data3_1)/np.sqrt(len(data3_1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean, std:  -0.00255278315035 0.000796258659538
mean, std:  -0.00607773982869 0.000788773392353
mean, std:  0.00328184708519 0.000782182135763
mean, std:  -0.0106432175811 0.00777833856286
mean, std:  0.00302278763124 0.0076577321529
mean, std:  -0.00762042994982 0.0105163138211


In [292]:
scaleZ, voltage, voltage_125_1, voltage_150_1

(128.33230649865476,
 24.026512677896338,
 119.33286771283258,
 138.41488735055043)

In [743]:
amps_all = list(amps_with_fb_spin_xz_100Vpp_93kHz_z_125Vpp_alternate_no_delay)+list(amps_with_fb_spin_xz_100Vpp_93kHz_z_125Vpp_alternate_no_delay_2)
# amps_all = list(amps_with_fb_spin_xz_100Vpp_93kHz_z_125Vpp_alternate_no_delay_2)
amps_all = np.array(amps_all)
len(amps_all)

5500

In [111]:
amps_spin_xz_117kHz_y_alternate_200Vpp_redo11 = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i) for i in tqdm(range((2500)))))


100%|██████████| 2500/2500 [00:10<00:00, 245.02it/s]

In [1508]:
amps_all = list(np.array(amps_no_fb_spin_xy_103kHz_z_200Vpp_alternate)*etha)
amps_all += list(np.array(amps_no_fb_spin_xy_103kHz_z_200Vpp_alternate_redo)*etha)
amps_all += list(np.array(amps_no_fb_spin_xy_103kHz_z_200Vpp_alternate_redo3)*etha) ### after RGA incident
amps_all += list(np.array(amps_no_fb_spin_xy_103kHz_z_200Vpp_alternate_redo4)*etha)  # after RGA incident

amps_all += list(np.array(amps_no_fb_spin_xy_93kHz_z_200Vpp_alternate)*etha)

# amps_all += list(np.array(amps_no_fb_spin_xy_93kHz_z_150Vpp_alternate)*etha*to_epsilon_150)
# amps_all += list(np.array(amps_no_fb_spin_xz_93kHz_z_150Vpp_alternate_5_delay)*etha*to_epsilon_150)
# amps_all += list(np.array(amps_no_fb_spin_xz_93kHz_z_150Vpp_alternate_6_delay)*etha*to_epsilon_150)

# amps_all = list(np.array(amps_no_fb_spin_xy_103kHz_z_200Vpp_both1)*etha)
# amps_all += list(np.array(amps_no_fb_spin_xy_103kHz_z_200Vpp_both2)*etha)
# amps_all += list(np.array(amps_no_fb_spin_xy_103kHz_z_200Vpp_both3)*etha)

amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp)*etha)
# amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo)*etha)  ## 4sec. files
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo2)*etha)
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo3)*etha)
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo4)*etha)
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo5[:-1])*etha)
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo6)*etha)
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo7)*etha)
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo8)*etha)
amps_all += list(np.array(amps_spin_xz_117kHz_y_alternate_200Vpp_redo9)*etha)

amps_all = np.array(amps_all)

In [112]:
amps_all = amps_spin_xz_117kHz_y_alternate_200Vpp_redo11*etha

In [113]:
etha = 1/scaleZ*327*1.6e-19*voltage
to_epsilon_200 = 1/(2*327*1.6e-19*voltage_200_1*(418)/(418+627))
to_epsilon_175 = 1/(2*327*1.6e-19*voltage_175_1*(418)/(418+627))
# to_epsilon_150 = 1/(2*327*1.6e-19*voltage_150_1*(418)/(418+627))
# to_epsilon_125 = 1/(2*327*1.6e-19*voltage_125_1*(418)/(418+627))

In [137]:
data = amps_all[::2,0]
data2 = amps_all[1::2,0]
data3 = data-0.66*data2

data_1 = np.abs(amps_all[::2,1])
data2_1 = np.abs(amps_all[1::2,1])
data3_1 = data_1+0.66*0.66*data2_1

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data))*2*2*10/3600, data, label='200Vpp-z-plus, xy-spin-50Vpp-103kHz')
ax.scatter(np.arange(len(data2))*2*2*10/3600, data2, label='200Vpp-z-minus, xy-spin-50Vpp-103kHz', alpha=0.6)
ax.scatter(np.arange(len(data3))*2*2*10/3600, data3, label='200Vpp-z, A parameter', alpha=0.6)

ax.plot(np.arange(0,140), np.arange(0,140)*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, electrode 1: mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)))
print('1st harmonic, electrode 2: mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))
print('1st harmonic, A parameter: mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data3)*to_epsilon_200, to_epsilon_200*np.std(data3)/np.sqrt(len(data3)))

print()
print('2nd harmonic, electrode 1: mean, std: ', np.mean(data_1), np.std(data_1)/np.sqrt(len(data_1)))
print('2nd harmonic, electrode 2: mean, std: ', np.mean(data2_1), np.std(data2_1)/np.sqrt(len(data2_1)))
print('2nd harmonic, B parameter: mean, std: ', np.mean(data3_1), np.std(data3_1)/np.sqrt(len(data3_1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, electrode 1: mean, std:  -3.0001001075e-17 2.11831927152e-18
1st harmonic, electrode 2: mean, std:  -6.12395638759e-17 2.68609095436e-18
1st harmonic, A parameter: mean, std:  1.04171110831e-17 1.46092272191e-18
1st harmonic, A parameter[epsilon]: mean, std:  0.00130348247762 0.000182803769105

2nd harmonic, electrode 1: mean, std:  2.74057696044e-16 3.67811879699e-18
2nd harmonic, electrode 2: mean, std:  5.35884201442e-16 7.57663054856e-18
2nd harmonic, B parameter: mean, std:  5.07488854192e-16 4.91806135762e-18


In [1510]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 29
range_pram = 0.02
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data3*to_epsilon_200, bins=n_bins, range=(-range_pram, range_pram), label='200Vpp, all frequencies')
# entries3, bins, patches = ax.hist(data3, bins=n_bins, range=(-range_pram, range_pram), label='150Vpp, 77kHz', alpha=0.6)
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1511]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']

for i,ee in enumerate([entries]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.03       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], " ,std (on the mean): ", popt[2]/np.sqrt(len(data)), " ,sigma: ", popt[2])
    print("std from the covariance matrix: ", np.sqrt(pcov[1,1]))
    print('\nNeutrality: ', 1/1.26e14*popt[2]/np.sqrt(len(data)))
    print('Factor needed for neutrality best limit: ', 1/1.26e14*popt[2]/np.sqrt(len(data))/1e-21)

mean:  4.77705272317e-05  ,std (on the mean):  5.88735140766e-05  ,sigma:  0.0065075875469
std from the covariance matrix:  4.98915234474e-05

Neutrality:  4.67250111719e-19
Factor needed for neutrality best limit:  467.250111719


In [1487]:
z_top_means = []
z_bottom_mean = []
z_top_std = []
z_bottom_std = []

bin_size = 10
bins = len(data)//bin_size
for i in range(bins):
    z_top_means.append(np.mean(data[i*bin_size:(i+1)*bin_size]))
    z_bottom_mean.append(np.mean(data2[i*bin_size:(i+1)*bin_size]))
    z_top_std.append(np.std(data[i*bin_size:(i+1)*bin_size]))
    z_bottom_std.append(np.std(data2[i*bin_size:(i+1)*bin_size]))
z_top_means = np.array(z_top_means)
z_top_std = np.array(z_top_std)
z_bottom_mean = np.array(z_bottom_mean)
z_bottom_std = np.array(z_bottom_std)

In [1488]:
z_combine_means = []
z_combine_std = []

bin_size = 50
bins = len(data)//bin_size
for i in range(bins):
    z_combine_means.append(np.mean(data3[i*bin_size:(i+1)*bin_size]))
    z_combine_std.append(np.std(data3[i*bin_size:(i+1)*bin_size]))
z_combine_means = np.array(z_combine_means)
z_combine_std = np.array(z_combine_std)/np.sqrt(bin_size)

In [1489]:
z_combine_means_2 = []
z_combine_std_2 = []

bin_size = 50
bins = len(data)//bin_size
for i in range(bins):
    z_combine_means_2.append(np.mean(data3_1[i*bin_size:(i+1)*bin_size]))
    z_combine_std_2.append(np.std(data3_1[i*bin_size:(i+1)*bin_size]))
z_combine_means_2 = np.array(z_combine_means_2)
z_combine_std_2 = np.array(z_combine_std_2)/np.sqrt(bin_size)

In [1490]:
len(z_combine_means), len(z_combine_std)

(243, 243)

In [1491]:
_,ax = plt.subplots(figsize=(9.4,4))
iteration = np.arange(len(z_combine_means))*10*bin_size*2*2/3600
ax.plot(iteration, iteration*0, 'r--')
top_err = z_top_std/np.sqrt(bin_size)
bottom_err = z_bottom_std/np.sqrt(bin_size)
A = z_top_means-z_bottom_mean*0.66
A_err = np.sqrt(top_err**2+top_err**2*0.66*0.66)
# ax.errorbar(iteration, z_top_means, yerr=top_err, fmt='o', label='top electrode')
# ax.errorbar(iteration, z_bottom_mean*0.66, yerr=bottom_err, fmt='o', label='bottom electrode')
# ax.errorbar(iteration, A, yerr=A_err, fmt='o', label='A parameter')
ax.errorbar(iteration, z_combine_means, yerr=z_combine_std, fmt='o', label='A parameter', alpha=0.3)
# ax.errorbar(iteration, z_combine_means_2, yerr=z_combine_std_2, fmt='o', label='A parameter', alpha=0.3)
ax.legend()
ax.set(xlabel='Measuring time [hr.]', ylabel='Measured force [N]')
# ax.errorbar(iteration, (z_top_means-z_bottom_mean*0.66)*to_epsilon, yerr=z_bottom_std/np.sqrt(10)*to_epsilon, fmt='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Measuring time [hr.]'), Text(0, 0.5, 'Measured force [N]')]

In [1492]:
A_epsion = z_combine_means*to_epsilon
A_err_epsilon = z_combine_std*to_epsilon
def calculate_pl(force_):
    chi2 = np.power((A_epsion-force_)/A_err_epsilon, 2)
    return np.sum(chi2)

In [1494]:
_,ax = plt.subplots()
pl_range = np.arange(-3e-4,3e-4,1e-5)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.plot(pl_range*100000, pl-np.min(pl), 'o--')
ax.plot(pl_range*100000, pl_range*0+2.7, 'r--')
ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, '$\\epsilon [10^5]$'), Text(0, 0.5, 'PL')]

In [1495]:
_,ax = plt.subplots()
pl_range = np.arange(-3e-4,3e-4,1e-5)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.plot(pl_range*100000, pl-np.min(pl), 'o--')
ax.plot(pl_range*100000, pl_range*0+2.7, 'r--')
ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')

pl_range_plus = np.arange(0,3e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--')

pl_range_minus = np.arange(-3e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--')

print(pl_range[np.argmin(pl)])
print(np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1]), np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

8.13151629364e-19
-9.00872437945e-05 9.64800137645e-05


In [1194]:
_,ax = plt.subplots()
pl_range_plus = np.arange(0,1e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--')
ax.plot(pl_range_plus*100000, pl_range_plus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-1.9999999999999239e-05, 8.5436025596824383e-05)

In [1042]:
_,ax = plt.subplots()
pl_range_minus = np.arange(-2e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--')
ax.plot(pl_range_minus*100000, pl_range_minus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-8.000000000000002e-05, -0.00017618157279836711)